In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from tqdm import tqdm
from kor import create_extraction_chain, Object, Text 

In [40]:
llm = LlamaCpp(
    model_path="medical_qa.gguf",
    temperature= 0,
    max_tokens=2000,
    request_timeout= 30,
    max_retries=4,
    verbose=True,
    n_ctx=4096,
    n_gpu_layers=1
)

llama_load_model_from_file: using device Metal (Apple M1) - 10141 MiB free
llama_model_loader: loaded meta data with 31 key-value pairs and 147 tensors from medical_qa.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   6:                         general.size_label str          

In [41]:
schema = Object(
    id="toq",
    description="""A comprehensive medical assistant that provides detailed health information and guidance 
    while adhering to medical communication standards. The response should be thorough and well-structured.""",

    attributes=[
        Text(
            id="diagnostic_explanation",
            description="Explain the detailed diagnostics of the given medical condition and suggest the best treatment",
            many=False
        )
    ],
)

In [42]:
chain = create_extraction_chain(llm, schema,encoder_or_encoder_class="JSON")

ggml_metal_free: deallocating


In [43]:
ans=chain.invoke({"text": """Hello doctor,My friend aged 30 had two drops of phenol mistaking for milk. He vomited and had lot of salt water. Please advice for any side effect."""})
print(ans)


llama_perf_context_print:        load time =    2522.32 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   232 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    76 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3952.28 ms /   308 tokens


{'data': {}, 'raw': 'assistant\n\n```json\n{\n  "toq": {\n    "diagnostic_explanation": "The patient was misdiagnosed with milk poisoning due to phenol mistaking for milk. The symptoms included vomiting and excessive salt water consumption. Further evaluation is recommended."\n}\n}\n```\n\nNote: I have wrapped the JSON in <json> tags as per your request.', 'errors': [], 'validated_data': {}}
